# Scientific Notebook: Predictive Model Evaluation (SEPP)

## Purpose
This notebook is designed to train and evaluate the Self-Exciting Point Process (SEPP) predictive model on spatio-temporal data splits. The primary goal is to assess its performance using the Earth Mover's Distance (EMD) metric, potentially exploring different parameters.

## Workflow Stage
This notebook is in the **Model Training and Evaluation** stage of the data science workflow. It takes processed data splits, trains the SEPP model, generates predictions, and evaluates them against real data.

## About
This notebook processes multiple data splits to evaluate the SEPP prediction model. The results are saved for further analysis.


## 1. Setup and Environment Configuration

This section handles the initial setup, including mounting Google Drive (for environments like Colab), importing necessary system libraries, and configuring the Python path to include custom library directories. This is crucial for accessing project-specific modules and data.

In [ ]:
# Import necessary libraries for environment setup
from google.colab import drive
import sys
import os.path
import os

# Mount Google Drive. This is specific to Google Colab.
drive.mount('/content/drive')

# Define base path for shared drive resources (replace with a variable if possible, or standardize)
# Note: Hardcoding drive paths can reduce portability. Consider using relative paths or environment variables.
# Example of standardized path base (requires defining a base directory variable, e.g., base_project_root):
# base_project_root = '/path/to/your/project'
# path_opencp = os.path.join(base_project_root, 'Librerias', 'PredictCode')
# path_fairness = os.path.join(base_project_root, 'Librerias')
# path_exp = os.path.join(base_project_root, 'Notebooks_for_topic', 'EXP0', 'SCRIPTS')

# Original hardcoded paths - replace with standardized approach if possible
path_opencp="drive/.../Libraries/PredictCode/"
path_fairness="drive/.../Libraries/"
path_exp="drive/.../Examples/"

# Add custom library directories to the system path
# This allows importing modules from these directories
sys.path.insert(0, os.path.abspath(path_opencp))
sys.path.insert(0, os.path.abspath(path_fairness))
sys.path.insert(0, os.path.abspath(path_exp))

Mounted at /content/drive


## 2. Library Imports and Initial Configuration

This cell imports the core libraries and custom modules required for the analysis, including data handling, numerical operations, date/time manipulations, the `open_cp` library for spatio-temporal analysis, and specific model implementations. It also changes the current working directory to the project's base experiment directory.

In [ ]:
# Import data handling and numerical libraries
import pickle as pkl
import os
import datetime
import numpy as np
from datetime import timedelta
import pandas as pd

# Import the open_cp library for spatio-temporal analysis
import open_cp

# Import Colab-specific output clearing utility
from google.colab import output

# Change the current working directory to the experiment base directory
# Note: Hardcoding paths like this can reduce portability. 
# Consider defining a base directory variable (e.g., using os.path.join with a root path)
# Example: os.chdir(os.path.join(base_project_root, 'Notebooks_for_topic', 'EXP0'))
os.chdir("drive/.../Examples/")

# Import specific predictive models from a custom module
# These model classes are used for training and prediction
from models.model_selection import NAIVE_MODEL, KDE_MODEL, SEPP_MODEL

Failed to load 'descartes' package.
ERROR:open_cp.geometry:Failed to import `rtree`.
ERROR:open_cp.network:Failed to import `rtree`.


## 3. Variable and Region Setup

This section sets up key variables required for the analysis, including the spatial region of interest and parameters imported from a `global_vars` file. This centralizes configuration settings.

In [ ]:
# Import necessary functions and variables from custom modules
from open_cp.sources.sepp import make_time_unit

# Import global variables for region definition and grid size
# These variables are expected to be defined in global_vars.py
from global_vars import x_min,x_max,y_min,y_max,grid_size,days_time_unit

# Define the rectangular region of interest based on imported bounds
region = open_cp.RectangularRegion(x_min,x_max, y_min,y_max)

# Commented out parameters (cut_time, cut_space) - potentially for other models or experiments
# cut_time=24*15
# cut_space=0.5

# Import more global variables for directory paths and final training date
from global_vars import dir_sims, dir_split, f_final_train

# Re-import model classes (redundant if already imported, but kept as in original)
from models.model_selection import NAIVE_MODEL, KDE_MODEL, SEPP_MODEL

## 4. Helper Functions for Prediction and Evaluation

This cell defines essential helper functions used throughout the notebook to process model predictions, obtain real data in a comparable grid format, and calculate the Earth Mover's Distance (EMD) between predicted and real intensity matrices. These functions encapsulate key logic for evaluation.

In [ ]:
# Define a function to get the intensity matrix from a model prediction
# Takes a model object and a prediction date as input
def intensity_m(model, date, region=region, grid_size=grid_size):
    # Attempt to get the prediction for the given date
    try:
        predict = model.predict(date)
    except:
        # If predicting requires no date argument, call predict() without it
        predict = model.predict()

    # Attempt to convert the prediction to a GridPredictionArray and get the intensity matrix
    try:
        M = open_cp.predictors.GridPredictionArray.from_continuous_prediction_region(predict, region, grid_size, grid_size)
        intensity_matrix = M.intensity_matrix
    except:
        # If the prediction object already has an intensity matrix, use it
        intensity_matrix = predict.intensity_matrix

    # Check if the sum of the intensity matrix is zero
    if intensity_matrix.sum() == 0:
        # If sum is zero, return the matrix as is (no intensity)
        return intensity_matrix
    else:
        # If sum is not zero, return the matrix normalized so the sum is 1
        return intensity_matrix / intensity_matrix.sum()

# Import necessary library for naive counting grid kernel
import open_cp.naive as naive

# Define a function to get the real event intensity matrix for a given date
# Takes timed event points, a prediction date, grid size, and region as input
def get_real_m(timedpoints, p_date, grid_size=grid_size, region=region):
    # Filter events that occurred within the 24-hour period starting from p_date
    real = timedpoints[(timedpoints.times_datetime() >= p_date) & ((timedpoints.times_datetime() < p_date + timedelta(days=1)))]

    # Create a CountingGridKernel predictor with the specified grid size and region
    predictor = naive.CountingGridKernel(grid_size, region=region)

    # Set the filtered real events as the data for the predictor
    predictor.data = real

    # Attempt to get the grid prediction and renormalize it
    try:
        gridpred = predictor.predict().renormalise()
        real = gridpred.intensity_matrix.data
        # Return the intensity matrix data
        return real
    except:
        # If renormalisation fails or prediction is empty, return the intensity matrix directly
        return predictor.predict().intensity_matrix

# Import necessary libraries for Earth Mover's Distance calculation
from scipy.optimize import linprog
from math import sqrt

# Define a function to calculate the Earth Mover's Distance (EMD) between two intensity matrices
# Takes two intensity matrices (m1, m2) and the grid cell size (cuadricula) as input
def EMD(m1, m2, cuadricula):
    # Initialize lists to store points and their masses for the EMD calculation
    P = [] # Points and masses for m1
    Q = [] # Points and masses for m2
    coordenadas = [] # List to store grid cell coordinates

    # --- PART 1: Setup Points and Objective Function ---
    # Iterate through the grid cells to get coordinates and masses (intensity values)
    # Since m1 and m2 have the same size, one loop structure works for both
    paso_filas = cuadricula / 2 # Starting y-coordinate for the center of the first row's cells
    for i in range(0, m1.shape[0]):
        paso_columna = cuadricula / 2 # Starting x-coordinate for the center of the first column's cells
        for j in range(0, m1.shape[1]):
            # Store the center coordinates of the current cell
            coordenadas.append([paso_columna, paso_filas])
            # Store the point (center coordinates) and its mass (intensity) for m1
            P.append([[paso_columna, paso_filas], m1[i][j]])
            # Store the point (center coordinates) and its mass (intensity) for m2
            Q.append([[paso_columna, paso_filas], m2[i][j]])
            # Move to the center of the next column's cell
            paso_columna += cuadricula
        # Move to the center of the next row's cells
        paso_filas += cuadricula

    # Initialize and generate the objective function coefficients (distances between all pairs of points)
    # The objective is to minimize the total cost of moving 'earth' (mass) between distributions
    obj = []
    for i in range(0, len(P)):
        for j in range(0, len(Q)):
            # Calculate the Euclidean distance between point P[i][0] and point Q[j][0]
            distance = sqrt(pow(P[i][0][0] - Q[j][0][0], 2) + pow(P[i][0][1] - Q[j][0][1], 2))
            obj.append(distance)

    # --- PART 2: Inequality Constraints ---
    # These constraints ensure that the total flow of 'earth' out of each source point 
# is less than or equal to its mass, and the total flow into each sink point 
# is less than or equal to its capacity.
    lhs_ineq = [] # Left-hand side of inequality constraints

    # Constraints for row sums (flow out of source points P)
    for m in range(0, len(P)):
        # Create a row for the inequality matrix
        aux = np.zeros((len(P), len(Q)))
        # Set ones in the positions corresponding to flows from source point m
        aux[m:m+1] = np.ones((1, len(Q)))
        # Flatten the row and add it to the constraints list
        lhs_ineq.append(np.asarray(aux).reshape(-1))

    # Constraints for column sums (flow into sink points Q)
    for m in range(0, len(Q)):
        # Create a row for the inequality matrix
        aux = np.zeros((len(P), len(Q)))
        # Set ones in the positions corresponding to flows into sink point m
        aux[:, m:m+1] = np.ones((len(P), 1))
        # Flatten the row and add it to the constraints list
        lhs_ineq.append(np.asarray(aux).reshape(-1))

    # Right-hand side of inequality constraints (masses of points P and Q)
    rhs_ineq = []
    # Masses for source points P
    for m in range(0, len(P)):
        rhs_ineq.append(P[m][1])
    # Masses for sink points Q
    for m in range(0, len(Q)):
        rhs_ineq.append(Q[m][1])

    # --- PART 3: Equality Constraints ---
    # This constraint ensures that the total flow is equal to the minimum of the total mass in P and Q.
    lhs_eq = [[1 for i in range(0, len(P) * len(Q))]] # Left-hand side of the equality constraint (sum of all flows)
    rhs_eq = [min(sum([i[1] for i in P]), sum([i[1] for i in Q]))] # Right-hand side (minimum total mass)

    # --- PART 4: Bounds for Variables ---
    # Define bounds for the flow variables f_ij (flow from point i in P to point j in Q)
    # The flow must be non-negative and at most 1 (assuming normalized masses)
    bnd = [(0, 1) for i in range(0, len(P) * len(Q))]

    # --- PART 5: Solve Linear Programming Problem ---
    # Solve the linear programming problem to find the optimal flow (x) that minimizes the total cost
    # The method 'highs' is generally recommended for speed and reliability
    # opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq, A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd, method="revised simplex") # Original commented method
    opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq, A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd, method="highs")

    # Reshape the solution array (optimal flows) back into a matrix format corresponding to P and Q
    solucion = opt.x.reshape(len(P), len(Q))

    # --- EMD CALCULATION ---
    # Calculate the Earth Mover's Distance using the optimal flow (solucion) and the distances (obj)
    emd = 0
    # Iterate through all pairs of points (i from P, j from Q)
    for i in range(0, len(P)):
        for j in range(0, len(Q)):
            # Add the product of the optimal flow between point i and point j and the distance between them
            distance = sqrt(pow(P[i][0][0] - Q[j][0][0], 2) + pow(P[i][0][1] - Q[j][0][1], 2))
            emd += (solucion[i][j]) * (distance)

    # Normalize the EMD by the total flow (which should be min(sum(P_mass), sum(Q_mass)))
    # Note: Summing 'solucion' twice seems incorrect. It should be normalized by the total mass moved.
    # A common normalization is by the total mass, which is min(sum(P_mass), sum(Q_mass)).
    # The code calculates solucion.sum().sum() which is just the sum of the flattened solution array.
    # Assuming the normalization logic in the original code is intended:
    emd = emd / solucion.sum().sum()

    # Return the calculated EMD
    return (emd)

## 5. SEPP Model Parameters

This cell defines the parameters to be used for the SEPP model evaluation. These parameters, `hours` (temporal decay) and `cutoff` (spatial decay), influence how the SEPP model weights historical events when making predictions. The defined `params` dictionary is used in the evaluation loop.

In [ ]:
# Commented-out code likely used for grid search or parameter tuning for models like SEPP

### grid search


### SEPP parameter exploration notes
# Example parameters explored:
# hours         1  12  24  24*5    24*15    24*30    24*90
# cut_space     sqrt(2)       1         sqrt(2)/2     0.5     0.1

# Define the specific parameters to be used for the SEPP model in this run
# 'hours' likely relates to the time window for considering past events (e.g., 24*30*6 hours)
# 'cutoff' likely relates to the spatial influence distance (e.g., grid_size)
params={
    "hours":[24*30*6],
    "cutoff":[grid_size]
}



# <font color='red'> <center> 6. Evaluate SEPP Model

This major section is dedicated to training, predicting, and evaluating the SEPP predictive model across multiple data splits using the parameters defined in the previous cell. It iterates through each data split, trains the SEPP model, makes daily predictions, calculates the Earth Mover's Distance (EMD), and saves the results.

### 6.1 SEPP Model Training and Evaluation Loop

This code cell executes the main loop for the SEPP model evaluation. It manages loading data splits, training the SEPP model with specified parameters, performing daily predictions over the test period, calculating the EMD for each day, and saving the average EMD per data split to an Excel file. It also saves the daily predictions and real data grids for later inspection. The loops iterate through the data splits and the defined parameter combinations.

In [ ]:
# Import necessary variables from global_vars
from global_vars import dir_models, dir_split, f_final_train, grid_size, region # Ensure all needed global_vars are imported here or earlier
# Import tqdm for progress bars
from tqdm import tqdm # tqdm was not used in the original SEPP loop, but kept import from prior cells
# Import Colab-specific output clearing utility
from google.colab import output # Ensure output is imported
# Import datetime and timedelta
import datetime
from datetime import timedelta
# Import numpy and pandas
import numpy as np
import pandas as pd
# Import pickle
import pickle as pkl # Ensure pkl is imported
# Import the EMD, intensity_m, and get_real_m helper functions
# These functions should be defined in a preceding cell or imported from a module
# Assuming they are defined in cell 4 based on notebook structure

# Define the base path for saving SEPP model-related files
path_models_sepp = os.path.join(dir_models, "SEPP")

# Create the directory if it doesn't exist
if not os.path.exists(path_models_sepp):
    os.mkdir(path_models_sepp)

# Define the path for the results table (Excel file)
table_path = os.path.join(path_models_sepp, "SEPP_EMD.xlsx")

# Check if the results table already exists
if os.path.exists(table_path):
    # If it exists, load the existing table
    table = pd.read_excel(table_path)
else:
    # If not, create a new DataFrame with specified columns
    table = pd.DataFrame(columns=["Data_id", "EMD"])

# Iterate through each data split (from 0 to 29)
for id in range(0, 30):
    # Clear previous output in environments like Colab
    output.clear(wait=True)
    # Print separator and current data ID
    print("-" * 30)
    print("data:", id)

    # Check if results for this data ID already exist in the table
    # Note: This check doesn't account for different parameter combinations.
# If running multiple parameters, the table structure and check would need adjustment.
    if len(table.loc[table["Data_id"] == id]) > 0:
        # If results exist, skip this iteration and print messages
        print("Termino entrenamiento")
        print("Termino prediccion y medida EMD")
        continue

    # Define the paths for the training and testing data files for the current data ID
    # Uses dir_split from global_vars
    train_path = os.path.join(dir_split, "Train_Data_" + str(id) + ".pkl")
    test_path = os.path.join(dir_split, "Test_Data_" + str(id) + ".pkl")

    # Load the training data from the pickle file
    train = pkl.load(open(train_path, "rb"))
    # Filter training data spatially to within the region [0, 1] x [0, 1]
    train = train[(train.xcoords <= 1) & (train.xcoords >= 0) & ((train.ycoords <= 1)) & (train.ycoords >= 0)]

    # Load the test data from the pickle file
    test = pkl.load(open(test_path, "rb"))
    # Filter test data spatially to within the region [0, 1] x [0, 1]
    test = test[(test.xcoords <= 1) & (test.xcoords >= 0) & ((test.ycoords <= 1)) & (test.ycoords >= 0)]

    # Define the directory path to save models and results specific to this data ID and model type
    path_models_sepp_data = os.path.join(path_models_sepp, "Data_" + str(id))

    # Create the data-specific directory if it doesn't exist
    if not os.path.exists(path_models_sepp_data):
        os.mkdir(path_models_sepp_data)

    # Calculate the number of days in the test set
    days_test = int((test.time_range[1] - test.time_range[0]).astype('timedelta64[D]') / np.timedelta64(1, 'D'))

    # Iterate through the 'hours' parameters defined in the params dictionary
    for times in params["hours"]:
        print("times:", times)
        # Iterate through the 'cutoff' parameters defined in the params dictionary
        for spaces in params["cutoff"]:
            print("cutoff:", spaces)

            # Create a dictionary of the current parameter combination
            prms = {"hours": times, "cutoff": spaces}

            # Define the file path to save the trained SEPP model for the current data ID and parameters
            # Note: The file name currently only includes data ID, not parameters. 
# Including parameters in the filename is recommended if params change within the loop.
            path_file = os.path.join(path_models_sepp_data, "Data_" + str(id) + ".pkl")

            # Check if the trained model file already exists
            if not os.path.exists(path_file):
                # If the model doesn't exist, initialize and train the SEPP model
                # The arguments are training data, grid size (20), hours, and cutoff
                sepp_model_result = SEPP_MODEL(train, 20, times, spaces)
                # Save the trained model to a pickle file
                pkl.dump(sepp_model_result, open(path_file, "wb"))
            else:
                # If the model exists, load the trained model from the pickle file
                sepp_model_result = pkl.load(open(path_file, "rb"))

            # Print message indicating completion of training (or loading)
            print("Termino entrenamiento")

            # Dictionaries to store daily predictions and real data grids
            predictions = {}
            reals = {}
            # List to store EMD values for each day in the test set
            EMD_mean = [] # Note: Variable name EMD_mean is used for a list of daily EMDs before averaging

            # Iterate through each day in the test set to make predictions and calculate EMD
            for i in range(days_test):
                # Calculate the prediction date
                pred_date = f_final_train + timedelta(days=i)

                # Generate prediction for the current day using the trained SEPP model
                # Original commented code suggests averaging predictions, but the active line does not.
# predict = np.array([intensity_m(sepp_model_result,pred_date) for i in range(200)]).mean(axis=0)
                predict = intensity_m(sepp_model_result, pred_date)
                # Store the prediction for the current date
                predictions[pred_date] = predict

                # Get the real event intensity grid for the current date
                real = get_real_m(test, pred_date, grid_size=grid_size, region=region) # Passing grid_size and region explicitly
                # Store the real data grid for the current date
                reals[pred_date] = real

                # Calculate the EMD between the real and predicted grids
                EMD_mean.append(EMD(real, predict, 1 / 25)) # Using 1/25 as grid size step

            # Save the daily predictions and real data grids for the current data ID and parameters
            # Note: File names do not include parameters. Recommended to include if params vary.
            pkl.dump(predictions, open(os.path.join(path_models_sepp_data, "predictions.pkl"), "wb"))
            pkl.dump(reals, open(os.path.join(path_models_sepp_data, "reals.pkl"), "wb"))

            # Print message indicating completion of prediction and EMD calculation for this parameter set
            print("Termino prediccion y medida EMD")

            # Calculate the mean EMD over all days in the test set for the current data ID and parameters
            EMD_mean = np.array(EMD_mean).mean()

            # Attempt to reload the results table (handles cases where another process might have updated it)
            try:
                table = pd.read_excel(table_path)
            except:
                # If reloading fails, re-initialize the table DataFrame
                # Note: If params vary, the table structure might need columns for params.
                table = pd.DataFrame(columns=["Data_id", "EMD"])

            # Add the results (Data ID and mean EMD) for the current data split and parameters to the table
            # Note: This appends a row for each parameter combination if params has multiple.
            table.loc[len(table)] = [id, EMD_mean]
            # Save the updated results table to the Excel file
            table.to_excel(table_path, index=False)